In [18]:
import tweepy
consumer_key = "VqcxzBOXycA5eSY4DVh8gdbcN"
consumer_secret = "kLkViKhCJ9VaXAeuvNr2bEBgC5pcYDOi5lwh5vXuTtc0bpL9L7"
access_token = "400927565-O9z7W7ucgL7esDLkRg2e4JSTfiNSTixqlf2tmC7i"
access_token_secret = "Oe2ohU0flKGf1HUn0ZkVTXmJ2tA3IgXCjkyFgo9x75imF"

In [19]:
#creating the authentication object
auth= tweepy.OAuthHandler(consumer_key, consumer_secret)
#setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
#creating the API object while passing in auth information
api = tweepy.API(auth)

In [20]:
tweets = api.search("qatarairways OR qrsupport", lang='en',rpp=100)

In [30]:
# foreach through all tweets pulled
for tweet in tweets:
   # printing the text stored inside the tweet object
   print(tweet.user.screen_name,"Tweeted:",tweet.text)

reem101131 Tweeted: RT @qatarairways: Game on #Oman! We are the Official Airline Partner for three of Oman’s Ministry of Education sports programmes and the Om…
alpamaika Tweeted: @qatarairways 24 h from Bangkok to Rome is extremely long, arrived in Doha 40 min before the connection flight and… https://t.co/aaOOsXR5Rm
medshare Tweeted: RT @AirlinkFlight: @qatarairways is making a huge difference for #CycloneIdai survivors. They've donated a flight carrying @SavetheChildren…
AliHasa_n Tweeted: #Flights on time 2017

#Qantas 86.2%
#JapanAirlines 85.3%
#Singapore 84.1%
#AllNippon 83.8%
#QatarAirways 82.9%… https://t.co/nVbWR1git4
qrsupport Tweeted: @WrisJarrett Investigate further and revert back to you. Please send to tell-us@qatarairways.com.qa
Thank you. 2/2.
GulfTimes_QATAR Tweeted: #US govt should not restrict #QatarAirways from #flying to the country, say 3 #airlines
#aviation… https://t.co/cDRwq5BNUT
qrsupport Tweeted: @WrisJarrett We can't be apologetic enough for the inconvenienc

In [6]:
import pandas as pd


In [7]:
# import csv #Import csv


# # Open/create a file to append data to
# csvFile = open('result.csv', 'a')

# #Use csv writer
# csvWriter = csv.writer(csvFile)

In [9]:
for tweet in tweepy.Cursor(api.search,"qatarairways OR qrsupport", lang='en',rpp=100).items():

    # Write a row to the CSV file.
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    print(tweet.created_at, tweet.text)
csvFile.close()

2019-04-23 05:25:58 @qrsupport Can you check now if Oryx One Communications will be available on tomorrows flight QR839? It’s really important for me to now.
2019-04-23 05:24:10 @qatarairways I have the ability to recognize all ( things / humans) from head side rather than back / tail 😋.
2019-04-23 05:20:11 @djfogarty Hello, we are here for your assistance. Mind sharing your claim reference number to follow up with our r… https://t.co/6jzugosCda
2019-04-23 05:18:20 @qatarairways To the best...
2019-04-23 05:11:11 @jay_rayzon Thank you for dedicating a bit of your precious time to write to us. We're really happy and glad to hea… https://t.co/eFGHaJQ7Yp
2019-04-23 05:04:48 @jaehwan_noona We've responded to your DM. Thanks!
2019-04-23 05:04:02 RT @livingwellands1: Qatar Airways
The pride of Qatar an airline making big splash with there exceptional customer service. And modern flee…
2019-04-23 05:02:08 @ChristopherXVII @qatarairways We had trouble too. Homophobia, targeted seating ( same s

2019-04-23 01:39:21 The sales staff of Qatar airlines in pakistan is a great blackmailer and money maker by kick backs. https://t.co/UnerXwRDIO
2019-04-23 01:35:35 @odio_su_tela Sorry for the inconvenience, Frika. You may clear your internet cache and cookies for a better result… https://t.co/IpmRVbNERK
2019-04-23 01:33:25 @qatarairways Hi Qatar, I have been waiting for someone to call me back all morning to discuss why I havent been no… https://t.co/mKwo9lXCtU
2019-04-23 01:19:02 . @qatarairways teams up with @RollsRoyce to test #virtualreality training https://t.co/Ld5pzq1Y5W https://t.co/oddlCYd7PB
2019-04-23 01:11:03 RT @joshimanasi11: Lost my sports prosthesis in transit in Doha @qatarairways i have my matches from tomorrow and I won't be able to play j…
2019-04-23 00:59:31 RT @joshimanasi11: Lost my sports prosthesis in transit in Doha @qatarairways i have my matches from tomorrow and I won't be able to play j…
2019-04-23 00:51:31 @qatarairways Nice
2019-04-23 00:46:45 @qatarairw

2019-04-22 18:08:18 RT @joshimanasi11: Lost my sports prosthesis in transit in Doha @qatarairways i have my matches from tomorrow and I won't be able to play j…
2019-04-22 18:06:59 RT @jonostrower: Clear cold soak. #qatarairways #airbusa350 #latergram #kjfk #avgeek @ John F. Kennedy International Airport https://t.co/F…
2019-04-22 18:03:27 RT @qatarairways: His Highness the Amir and President of Rwanda witnessed the signing of an agreement in the airspace and two draft memoran…
2019-04-22 18:01:14 Life is...reconnecting Art 

@miaqatar @qatar_museums @reconnectingart @qatarairways @qatarliving qatarcalendar… https://t.co/XFRjdvxajn
2019-04-22 17:59:30 RT @qatarairways: His Highness the Amir and President of Rwanda witnessed the signing of an agreement in the airspace and two draft memoran…
2019-04-22 17:58:21 @qatarairways Airbus A350
2019-04-22 17:56:09 @beenishshafi @qrsupport @emirates Sorry to hear that, I can relate with your issues big time
2019-04-22 17:55:54 @AzamJamil53 @qatar

2019-04-22 16:19:00 Emojis for #jugglelikeapro #qatarairways #asroma @qatarairways @asromaen @ Malaysia https://t.co/lOulZXVtQg
2019-04-22 16:15:32 🔝⚽🎯 @ColonOficial's Tomás Sandoval's back-heel goal was voted this week's @qatarairways #CONMEBOLSudamericana Goal… https://t.co/lu8pGt3IM0
2019-04-22 16:14:36 @qatarairways rusty fork as well https://t.co/ZbSdLh2tIA
2019-04-22 16:11:18 RT @Investidea1: Long $VNO Vornado Realty $SLG SL Green Realty $VGR Vector Group #Douglaselliman #Qatar fund buys chunk of US$5.6b Manhatta…
2019-04-22 16:10:01 RT @susantran: #Qatar... the popular pronunciation online sounds like cutter. While I flew #QatarAirways, they pronounced it kat-tar. Debat…
2019-04-22 16:03:14 @mmbuthiah You're most welcome. We'd love to hear your thoughts, tell us about your experience through a short surv… https://t.co/qoUR2QmuMC
2019-04-22 16:01:11 @jktala Hi, Jere. Kindly submit the same online at the “Contact center” portal, for further check and assistance fr… https://t.co/ss

2019-04-22 13:42:33 @qrsupport Thanks, but part of the problem is they haven't made contact. I emailed again asking for an update on th… https://t.co/R01aaCCwKU
2019-04-22 13:41:46 @Limontere Hi Teresa, were you able to get this sorted? Please share the booking reference via DM should you still… https://t.co/YX8OktrFMe
2019-04-22 13:41:27 RT @qatarairways: Did you know: @HIAQatar was opened in 30 April 2014, and since then we have welcomed millions of passengers. In 2018, our…
2019-04-22 13:40:00 Did you know: @HIAQatar was opened in 30 April 2014, and since then we have welcomed millions of passengers. In 201… https://t.co/NFpJJoY2bE
2019-04-22 13:39:17 @fda66 Hi Franco, sorry for the late reply. We have 2 lounges at Barcelona Airport: AENA Miro Lounge: Terminal 1 /… https://t.co/FfPW2QdxDK
2019-04-22 13:38:09 @ramonbullard @IndiGo6E @TurkishAirlines @airindiain @EtihadAirways @emirates @qatarairways However, I didn’t know… https://t.co/zk3xUTYGbn
2019-04-22 13:34:05 @ramonbullard @In

2019-04-22 11:00:39 RT @SertanDjelal: @qatarairways Had the most awful experience with @qatarairways! Can’t believe they advertise a ‘5* service’ yet have such…
2019-04-22 11:00:13 @qrsupport Hello. I already did (over 2 months ago) but didn't get anything yet. Noting that I didn't get the package to begin with.
2019-04-22 10:59:52 RT @CraigStevens_11: Love the curves on the @Boeing B787! @qatarairways arriving into @manairport with a slightly slow shutter speed 😋
➖➖➖➖…
2019-04-22 10:59:46 @qrsupport Thank you for the response. I am not a passenger, my relative is. I will let her know.
2019-04-22 10:58:39 Emails now sent to Hardin, Amish, Nikunj and numerous others @qatarairways @MOI_QatarEn @VisitQatar - all failing m… https://t.co/llW7KSkUi3
2019-04-22 10:58:02 RT @CraigStevens_11: Love the curves on the @Boeing B787! @qatarairways arriving into @manairport with a slightly slow shutter speed 😋
➖➖➖➖…
2019-04-22 10:55:18 @qrsupport Thank you for your kind help and support.  I really ap

2019-04-22 08:29:22 @qrsupport Is it possible to upgrade economy (S) tickets to business class on DOH - BKK QR834 on 13 August?
2019-04-22 08:22:51 @qatarairways your employees in the Islamabad office never pick up the phone. I've tried both numbers available and… https://t.co/5Uv01bg7NR
2019-04-22 08:22:48 @Finnair @qatarairways @British_Airways @cathaypacificUS @Qantas @AmericanAir @Iberia_en Thank you to Qantas, Ameri… https://t.co/bURBrztGfA
2019-04-22 08:19:08 RT @CraigStevens_11: Love the curves on the @Boeing B787! @qatarairways arriving into @manairport with a slightly slow shutter speed 😋
➖➖➖➖…
2019-04-22 08:16:30 @qatarairways Good
2019-04-22 08:16:01 RT @jay_rayzon: Nice to see that some airlines still do care about pax comfort... even in a A320 for a short haul flight. Thanks @qatarairw…
2019-04-22 08:15:21 RT @ASRomaEN: Retweet if Stephan El Shaarawy was your #ASRoma @QatarAirways Man of the Match against Inter https://t.co/IMnKCcGdfK
2019-04-22 08:14:06 @CraigStevens_11 @

2019-04-22 01:50:28 @qrsupport We fly out of Colombo this evening. Is this still your advice? Any updates?
2019-04-22 01:49:58 RT @TripleC86: @qatarairways @qrsupport I have still not heard anything from the airline about rescheduling the cancelled #QR905 flight!
2019-04-22 01:45:27 @qatarairways @qrsupport I have still not heard anything from the airline about rescheduling the cancelled #QR905 flight!
2019-04-22 01:37:13 #QatarAirways can you help please, the flight from Melbourne was cancelled earlier, no one at the airport to talk to
2019-04-22 01:24:47 @oscargopa @qrsupport I am in the same situation and have still not heard anything!
2019-04-22 00:52:23 RT FiveGeckos "RT SertanDjelal: Awful service from qatarairways HKT-DOH-LHR. Such incompetent &amp; rude staff misinfor… https://t.co/fu8IlDQBo6
2019-04-22 00:33:58 @vincentrosella We have reached out to you girlfriend, Vicente. Please check your DM. Thanks.
2019-04-22 00:28:53 RT @SertanDjelal: Awful service from @qatarairways HKT

In [49]:
x = tweets

type(x)

tweepy.models.SearchResults

In [50]:
import json

# pd.read_json(json.dumps(x._json))
dict(x._json)

AttributeError: 'SearchResults' object has no attribute '_json'

anything you can do with a dict you can do with json
easier to parse 

In [53]:
for x in tweets:
    print(x.id)

1120743556589785088
1120743131249385472
1120742787778060289
1120742384248270848
1120742196456624129
1120742013991903237
1120741726891778048
1120741627335589888
1120741437757362179
1120741329129099264
1120741145594736640
1120741116830208000
1120740836948336640
1120740274014162949
1120740158138138629


X is an object in that one tweet.

In [52]:
tweets

[Status(_api=<tweepy.api.API object at 0x1164257f0>, _json={'created_at': 'Tue Apr 23 17:37:40 +0000 2019', 'id': 1120743556589785088, 'id_str': '1120743556589785088', 'text': 'RT @qatarairways: Can you guess which aircraft this tail belongs to? #QatarAirways https://t.co/LOPQSgiLVA', 'truncated': False, 'entities': {'hashtags': [{'text': 'QatarAirways', 'indices': [69, 82]}], 'symbols': [], 'user_mentions': [{'screen_name': 'qatarairways', 'name': 'Qatar Airways', 'id': 14589119, 'id_str': '14589119', 'indices': [3, 16]}], 'urls': [], 'media': [{'id': 1120305302354706432, 'id_str': '1120305302354706432', 'indices': [83, 106], 'media_url': 'http://pbs.twimg.com/media/D4wfvKLWwAAaPWz.jpg', 'media_url_https': 'https://pbs.twimg.com/media/D4wfvKLWwAAaPWz.jpg', 'url': 'https://t.co/LOPQSgiLVA', 'display_url': 'pic.twitter.com/LOPQSgiLVA', 'expanded_url': 'https://twitter.com/qatarairways/status/1120698820411654144/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize':

Objective is to convert JSON file to DataFrame